#### Meaningful description 

In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Import and setup the tensorflow session
import tensorflow as tf

# Sanity check to ensure we are on correct tensorflow version
print("Using Tensorflow v. %s"%tf.__version__); assert tf.__version__ == "2.4.1";

# Setup a strategy to distribute resources and training
# across all available GPUs on the system.
strategy = tf.distribute.MirroredStrategy()

# Check number of GPUs used by the distribution scope.
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Using Tensorflow v. 2.4.1
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')
Number of devices: 3


In [2]:
# Import tf contrib modules
import tensorflow_probability as tfp

# Import Keras modules
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint

# Import from custom modules
from data_utils.mg_sg_generator import strip_file
from data_utils.mg_sg_generator import get_dataset_for, splice_spectrogram_patches
from data_utils.mg_sg_generator import prepare_dataset_for_training, prepare_dataset_for_evaluation

from data_utils.sg_preprocessor import spectrogram2audio
from data_utils.sg_preprocessor import inverse_normalize_db_0_1, FNCOLS 

# Import custom tensorflow utilities
from tf_extensions.tf_custom.models import GaussianBetaVAE
from tf_extensions.tf_custom.models import make_cnn_vae_encoder, make_dense_vae_decoder

# Common utilities
import numpy as np

# Display utilities
import matplotlib.pyplot as plt
import IPython.display as ipd

# Pathing (using absolute paths for now)
BASE_DATA_PATH = "/home/sbol13/sbol_data"
MW_SAVE_PATH   = BASE_DATA_PATH + "/model_weights"

#### Utility functions 

In [3]:
def sample_mvn(x_mu, x_logvar):
    x_sig = np.exp(x_logvar)
    batch = x_sig.shape[0]
    mvn = tfp.distributions.MultivariateNormalDiag(
        loc=x_mu, scale_diag=x_sig
    )
    return mvn.sample(shape=[batch]).numpy()

### Prepare and Setup Data
Construct and/or load the datasets we need for the various experiments:
- Linear Spectrogram <b>(r128xc128)</b>, latent dim: 256
- Linear Spectrogram <b>(r512xc128)</b>, latent dim: 256
- Linear Spectrogram <b>(r512xc128)</b>, latent dim: 256x4 (2048)

Where rNUM indicated the FFT bins (i.e. the FFT-size) used to compute the spectrograms we are attempting to predict. cNUM is simply the STFT columns, which are a direct result of the <i>hop_size</i> we use.  


#### Globals for all experiments: 
Some variables are held constant throughout the different models under experimentation.
We use the block below to define these, such that we can re-use them throughout the notebook.

In [4]:
# Convenience lambda(s) to retrieve information
# about a relevant .tfrecords file (nexamples, filepath)
TFR_TRAIN      = lambda tfr_dict: tfr_dict["train"]
TFR_VALIDATION = lambda tfr_dict: tfr_dict["validation"]
TFR_TEST       = lambda tfr_dict: tfr_dict["test"]
TFR_PATH       = lambda tfr_tuple: tfr_tuple[-1]

# Define a small lambda to compute the columns
# to use for each nn input. TODO: FINISH
NCOLS = lambda hl: FNCOLS(hl)

# Define a small lambda to compute the rows
# to use for each nn input.
NROWS = lambda nfft: nfft//2 + 1

# Define the train-size percentage.
# This value indicates how much of the dataset is reserved for training. 
# The remaining is split evenly into validation/testing.
# (i.e. a split could be 70-15-15)
TRAIN_SIZE = 0.7

# Define a flag to indicate whether we should
# recompute all the datasets.
DS_OVERWRITE = True

# Define the beta-value to use for tuning the VAE(s).
VAE_BETA = 1.

# Define the learning rate to apply across
# all training conditions.
BASE_LR = 2e-04 

# Define the optimizer to use for training across
# all conditions.
OPTIMIZER = optimizers.Adam(lr=BASE_LR)

# Define batch size to use for training.
BATCH_SIZE = 100

# Define a lambda to retrieve the step size(s).
STEPS_PER_EPOCH = lambda tfr_tuple: tfr_tuple[0] // BATCH_SIZE

# Lambda to retrieve savepath to model weights
MW_SAVEPATH_FN = lambda meta_dict:\
    f'{MW_SAVE_PATH}/{strip_file(TFR_PATH(TFR_TRAIN(meta_dict)))}.h5'

# Define a function to return a callback for model saving. 
# (setup saving weights incrementally)
def CHECKPOINT_CALLBACK(meta_dict, save_step=20):
    # Compute save frequency
    SF = STEPS_PER_EPOCH(TFR_TRAIN(meta_dict)) * save_step 

    # Make callback instance
    CHECKPOINT = ModelCheckpoint(
        MW_SAVEPATH_FN(meta_dict), 
        monitor           = 'ELBO', 
        save_freq         = SF,
        save_best_only    = True,
        save_weights_only = True,
        mode              ='auto',
        verbose           = 1
    )
    return CHECKPOINT

#### Approach 1 (AP1): Linspect r128xc128, latent dim 256

In [5]:
# Define the FFT-size.
ap1_nfft = 256

# Define the hop_length (overlap)
# in samples.
ap1_overlap = ap1_nfft // 2

# Define the latent dimension size.
ap1_latent_dim = 256

# Fetch a dataset that fullfils the 
# requirements for this condition.
ap1_fout = get_dataset_for(
    nfft               = ap1_nfft,
    overlap            = ap1_overlap,
    train_size         = TRAIN_SIZE, 
    overwrite_existing = DS_OVERWRITE
)

# Prepare the input/output shape(s).
ap1_inout_shape_train = (NROWS(ap1_nfft), TFR_TRAIN(ap1_fout)[2])

# Fetch a deep learning model (VAE) 
# for this condition.
#with strategy.scope():
ap1_vae = GaussianBetaVAE(
    N                   = TFR_TRAIN(ap1_fout)[0],
    M                   = BATCH_SIZE,
    beta                = VAE_BETA,
    input_dim           = ap1_inout_shape_train,
    latent_dim          = ap1_latent_dim,
    create_encoder_func = make_cnn_vae_encoder,
    create_decoder_func = make_dense_vae_decoder
)

# Compile the model and make ready for training.
ap1_vae.custom_compile(optimizer=OPTIMIZER)

In [8]:
# Prepare dataset(s) for training.
ap1_ds_train = prepare_dataset_for_training(
    filename     = TFR_PATH(TFR_TRAIN(ap1_fout)),
    batch_size   = BATCH_SIZE,
    cast_to_type = tf.float64
)

ap1_ds_validation = prepare_dataset_for_training(
    filename     = TFR_PATH(TFR_VALIDATION(ap1_fout)),
    batch_size   = BATCH_SIZE,
    cast_to_type = tf.float64
)

# Train the vae model.
ap1_history = ap1_vae.fit(
    ap1_ds_train,
    steps_per_epoch  = STEPS_PER_EPOCH(TFR_TRAIN(ap1_fout)),
    validation_data  = ap1_ds_validation,
    validation_steps = STEPS_PER_EPOCH(TFR_VALIDATION(ap1_fout)),
    epochs           = 100,
    callbacks        = [CHECKPOINT_CALLBACK(ap1_fout)]
)

Epoch 1/100


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument:  Type mismatch between parsed tensor (double) and dtype (float)
	 [[{{node ParseTensor_1}}]]
	 [[IteratorGetNext]]
  (1) Invalid argument:  Type mismatch between parsed tensor (double) and dtype (float)
	 [[{{node ParseTensor_1}}]]
	 [[IteratorGetNext]]
	 [[IteratorGetNext/_20]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_83350]

Function call stack:
train_function -> train_function


#### Evaluate AP1 model 

In [ ]:
train_meta = TFR_TRAIN(ap1_fout)
batch_div = train_meta[0] // train_meta[1]
ap1_ds_test = prepare_dataset_for_evaluation(
    filename     = TFR_PATH(TFR_TRAIN(ap1_fout)),
    batch_size   = batch_div,
    cast_to_type = tf.float64
)
ds_iter = iter(ap1_ds_test)

In [ ]:
mg_, _ = ds_iter.get_next()
mu_x, logvar_x, _, _ = ap1_vae.predict(mg_)
print(mu_x.shape, logvar_x.shape)

In [ ]:
specgram = sample_mvn(mu_x, logvar_x)
print(specgram.shape)

In [ ]:
specgram_patches_np = np.reshape(specgram, newshape=(batch_div, NROWS(ap1_nfft), -1))
specgram_patches = [specgram_patches_np[p, ...] for p in range(batch_div)]
specgram_spliced = splice_spectrogram_patches(specgram_patches, NCOLS(ap1_overlap))
print(specgram_spliced.shape)

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(10, 10))
axs.set_xticks([]); axs.set_yticks([]);
axs.imshow(
    np.flipud(specgram_spliced), 
    aspect="auto", 
    cmap="Spectral_r", 
    interpolation="bicubic"
)

In [ ]:
y = spectrogram2audio(
    specgram_spliced, 
    ap1_overlap, 
    True
)
print(y.shape)
ipd.display(ipd.Audio(y, rate=22050))

In [ ]:
print(MW_SAVEPATH_FN(ap1_fout))